In [3]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from umap import UMAP
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk

ImportError: cannot import name 'UMAP' from 'umap' (e:\Documents\Data Science\Projets\Python\envs\env-nlp-trip-advisor\Lib\site-packages\umap\__init__.py)

In [ ]:
# Télécharger les ressources NLTK nécessaires
nltk.download('punkt')

In [ ]:
# Charger les données
df = pd.read_csv('avis_restaurants.csv')

In [ ]:
# Prétraitement des avis
df['tokens'] = df['avis'].apply(lambda x: word_tokenize(x.lower()))

# Entraîner le modèle Word2Vec
model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
# Fonction pour obtenir le vecteur moyen d'un avis
def get_avg_vector(tokens):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(100)
    return np.mean(vectors, axis=0)

# Calculer les vecteurs moyens pour chaque restaurant
df['avg_vector'] = df['tokens'].apply(get_avg_vector)

# Agréger les vecteurs par restaurant
restaurant_vectors = df.groupby('restaurant')['avg_vector'].apply(lambda x: np.mean(list(x), axis=0))

In [ ]:
# Réduction de dimensionnalité avec UMAP
umap = UMAP(n_components=2, random_state=42)
restaurant_coords = umap.fit_transform(np.array(restaurant_vectors.tolist()))

# Visualisation
plt.figure(figsize=(12, 8))
plt.scatter(restaurant_coords[:, 0], restaurant_coords[:, 1])

for i, restaurant in enumerate(restaurant_vectors.index):
    plt.annotate(restaurant, (restaurant_coords[i, 0], restaurant_coords[i, 1]))

plt.title("Représentation des restaurants basée sur les avis (UMAP)")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()
